In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random
import ipaddress  # For advanced IP address manipulation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Data Simulation

In [2]:
def generate_synthetic_network_data(num_samples=10000, anomaly_ratio=0.01):
    """
    Generates a synthetic dataset of network connections. 
    A small fraction (anomaly_ratio) simulate possible data exfil (anomalies).
    """
    random.seed(42)
    np.random.seed(42)
    
    data = {
        'src_ip': [],
        'dst_ip': [],
        'bytes_out': [],
        'bytes_in': [],
        'hour_of_day': [],
        'label': []  # 0 = normal, 1 = potential exfil/anomaly
    }
    
    num_anomalies = int(num_samples * anomaly_ratio)
    
    for i in range(num_samples):
        hour = np.random.randint(0, 24)
        is_anomaly = (i < num_anomalies)
        data['label'].append(int(is_anomaly))
        
        if is_anomaly:
            # Potential exfil: large bytes_out, traffic to an external global IP
            src_ip = f"10.{np.random.randint(0,255)}.{np.random.randint(0,255)}.{np.random.randint(0,255)}"
            dst_ip = f"{np.random.randint(100,255)}.{np.random.randint(0,255)}.{np.random.randint(0,255)}.{np.random.randint(0,255)}"
            bytes_out = np.random.randint(500000, 2000000)  # large data transfer
            bytes_in  = np.random.randint(0, 1000)
        else:
            # Normal traffic stays in private range
            src_ip = f"10.{np.random.randint(0,255)}.{np.random.randint(0,255)}.{np.random.randint(0,255)}"
            dst_ip = f"10.{np.random.randint(0,255)}.{np.random.randint(0,255)}.{np.random.randint(0,255)}"
            bytes_out = np.random.randint(0, 50000)
            bytes_in  = np.random.randint(0, 50000)

        data['src_ip'].append(src_ip)
        data['dst_ip'].append(dst_ip)
        data['bytes_out'].append(bytes_out)
        data['bytes_in'].append(bytes_in)
        data['hour_of_day'].append(hour)
    
    return pd.DataFrame(data)

df = generate_synthetic_network_data(num_samples=10000, anomaly_ratio=0.01)
df.head()


,src_ip,dst_ip,bytes_out,bytes_in,hour_of_day,label
0,10.179.92.14,206.71.188.20,1603462,121,6,1
1,10.214.74.202,187.116.99.103,778167,130,18,1
2,10.52.1.87,137.129.191.187,1784372,160,21,1
3,10.57.21.252,188.48.218.58,1933257,475,11,1
4,10.14.189.189,150.107.54.243,1055839,504,15,1


# Feature Engineering

### IP Feature Extraction Function

In [19]:
def extract_ip_features(ip_str):
    """
    Convert an IP string into a vector of features using `ipaddress` library.
    
    Rationale behind each feature:
      - Checking whether an IP is private, global, loopback, etc., can help 
        detect unusual or out-of-policy traffic in anomaly detection use cases.
      - Identifying the IP version is vital for IPv6 vs. IPv4 scenarios, which 
        can differ in routing and security contexts.
      - The integer representation often feeds machine learning models better 
        than a raw string, allowing potential numeric patterns (e.g., subnets).
        
    Returns a list of numeric features in this order:
      [
        is_private, 
        is_global,
        is_loopback,
        is_link_local,
        is_reserved,
        is_multicast,
        ip_version,
        integer_representation
      ]
    """
    # Convert the string to an ipaddress object (handles both IPv4Address and IPv6Address).
    # Why: We need a structured representation to query properties like is_private, is_loopback, etc.
    ip_obj = ipaddress.ip_address(ip_str)
    
    # is_private: 1 if IP is in an RFC-defined private range (e.g. 10.x, 192.168.x).
    # Why: Traffic to/from private IPs often indicates internal-only communication. 
    #      A sudden spike in external (non-private) IP usage could signal data exfiltration or suspicious activity.
    is_private = int(ip_obj.is_private)
    
    # is_global: 1 if IP is a globally routable address.
    # Why: Important for anomaly detection—traffic to global IPs might be suspicious 
    #      if the environment typically restricts external connections or uses VPN tunnels.
    is_global = int(ip_obj.is_global)
    
    # is_loopback: 1 if IP is in the loopback range (127.x.x.x for IPv4).
    # Why: Loopback addresses typically mean local traffic. If loopback traffic appears 
    #      unexpectedly in logs, it might be a misconfiguration or malicious tunneling.
    is_loopback = int(ip_obj.is_loopback)
    
    # is_link_local: 1 if IP is in the link-local range (169.254.x.x).
    # Why: Link-local addresses are auto-configured, often used in internal subnets 
    #      without a DHCP server. Unusual link-local usage might signal misconfigurations.
    is_link_local = int(ip_obj.is_link_local)
    
    # is_reserved: 1 if IP is in a reserved block (non-routable or set aside for special use).
    # Why: Reserved addresses usually shouldn't appear in normal production traffic. 
    #      Their presence could indicate an error or nefarious testing by an attacker.
    is_reserved = int(ip_obj.is_reserved)
    
    # is_multicast: 1 if IP is in the multicast address range (224.0.0.0 to 239.255.255.255 for IPv4).
    # Why: Multicast is uncommon in many environments. Unexpected multicast traffic 
    #      can be a clue for suspicious network patterns or advanced tunneling.
    is_multicast = int(ip_obj.is_multicast)
    
    # ip_version: 4 or 6. IPv6 has different security and routing considerations than IPv4.
    # Why: Knowing the version helps handle logs differently and might impact how 
    #      anomaly thresholds are set—IPv6 can use huge address spaces or unique subnets.
    ip_version = ip_obj.version
    
    # Convert IP to an integer. For IPv4, it's a 32-bit number; for IPv6, 128-bit.
    # Why: ML models often prefer numeric input. It can also reveal patterns 
    #      (e.g., close numeric ranges may map to the same subnet).
    ip_int = int(ip_obj)
    
    return [
        is_private,
        is_global,
        is_loopback,
        is_link_local,
        is_reserved,
        is_multicast,
        ip_version,
        ip_int
    ]


### Add new features to full dataframe

In [4]:
def preprocess_dataframe(df):
    """
    1. Extract advanced IP features for both src_ip and dst_ip using `ipaddress`.
    2. Concatenate them with numeric columns (bytes_out, bytes_in, hour_of_day).
    3. Scale features using MinMaxScaler for improved training stability.
    4. Return a NumPy array of scaled features and the fitted scaler.
    """
    # Extract IP-based features for src_ip
    src_ip_features = df['src_ip'].apply(extract_ip_features).tolist()
    # Each element is an 8-dimensional feature vector from above
    
    # Extract IP-based features for dst_ip
    dst_ip_features = df['dst_ip'].apply(extract_ip_features).tolist()
    
    # Convert lists to numpy arrays
    src_ip_array = np.array(src_ip_features, dtype=float)
    dst_ip_array = np.array(dst_ip_features, dtype=float)
    
    # Combine with numeric columns
    numeric_cols = ['bytes_out', 'bytes_in', 'hour_of_day']
    numeric_data = df[numeric_cols].values.astype(float)
    
    # Final feature matrix
    X = np.hstack([src_ip_array, dst_ip_array, numeric_data])
    
    # Scale
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, scaler

# Preprocess the dataframe
X_scaled, scaler = preprocess_dataframe(df)
print("Shape of scaled input:", X_scaled.shape)


Shape of scaled input: (10000, 19)


# Build and Train the Autoencoder

### Dataset class

In [5]:
class NetworkDataset(Dataset):
    def __init__(self, features):
        self.features = torch.from_numpy(features).float() # Converts the NumPy array (features) to a float tensor so PyTorch can handle it
        
    def __len__(self):
        return self.features.shape[0] # Returns the dataset size (number of samples)
    
    def __getitem__(self, idx):
        return self.features[idx] # Retrieves one sample (feature vector) from the dataset by index

In [ ]:
class NetworkDataset(Dataset):
    """
    A lightweight PyTorch Dataset that wraps a NumPy array of features.
    """

    def __init__(self, features):
        """
        Converts the NumPy array (features) to a float tensor so PyTorch can handle it.
        """
        self.features = torch.from_numpy(features).float()
        
    def __len__(self):
        """
        Returns the dataset size (number of samples).
        """
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        """
        Retrieves one sample (feature vector) from the dataset by index.
        """
        return self.features[idx]


### Train/val/test split

In [6]:
X_train, X_temp = train_test_split(X_scaled, test_size=0.2, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)

train_dataset = NetworkDataset(X_train)
val_dataset   = NetworkDataset(X_val)
test_dataset  = NetworkDataset(X_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)
print("Test shape:", X_test.shape)

Train shape: (8000, 19)
Val shape: (1000, 19)
Test shape: (1000, 19)


### Define the Autoencoder

In [8]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim=16):
        """
        Simple fully-connected Autoencoder:
          - Encoder -> compress input_dim -> latent_dim
          - Decoder -> reconstruct input_dim from latent_dim
        """
        super(Autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

### Training loop

In [9]:
model = Autoencoder(input_dim=X_train.shape[1], latent_dim=16)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_autoencoder(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        
        for batch_features in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_features)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_features in val_loader:
                outputs = model(batch_features)
                loss = criterion(outputs, batch_features)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {total_loss/len(train_loader):.6f}, "
              f"Val Loss: {val_loss/len(val_loader):.6f}")

train_autoencoder(model, train_loader, val_loader, epochs=10)


Epoch 1/10, Train Loss: 0.047325, Val Loss: 0.014266
Epoch 2/10, Train Loss: 0.010884, Val Loss: 0.008656
Epoch 3/10, Train Loss: 0.003454, Val Loss: 0.001188
Epoch 4/10, Train Loss: 0.000329, Val Loss: 0.000517
Epoch 5/10, Train Loss: 0.000216, Val Loss: 0.000462
Epoch 6/10, Train Loss: 0.000187, Val Loss: 0.000398
Epoch 7/10, Train Loss: 0.000157, Val Loss: 0.000332
Epoch 8/10, Train Loss: 0.000126, Val Loss: 0.000289
Epoch 9/10, Train Loss: 0.000099, Val Loss: 0.000216
Epoch 10/10, Train Loss: 0.000081, Val Loss: 0.000219


# Anomaly Scoring and Thresholding

### Compute Reconstruction Error

In [14]:
def compute_reconstruction_errors(model, dataloader):
    model.eval()
    errors = []
    
    with torch.no_grad():
        for batch_features in dataloader:
            reconstruction = model(batch_features)
            # MSE per sample
            batch_loss = (reconstruction - batch_features).pow(2).mean(dim=1)
            errors.append(batch_loss.cpu().numpy())
    
    errors = np.concatenate(errors)
    return errors

# 1. Compute reconstruction errors on the test set
test_errors = compute_reconstruction_errors(model, test_loader)

# 2. Choose a threshold (e.g., 95th percentile)
threshold = np.percentile(test_errors, 95)

# 3. Flag anomalies (1) if the error exceeds threshold
anomaly_flags = (test_errors > threshold).astype(int)

# Prepare dataframe for viewing anomalies

### Reconstruct test dataframe

In [15]:
X_temp_df = df.iloc[len(X_train):].reset_index(drop=True)  # combined val+test portion
split_idx = len(X_val)  # the portion for validation
df_test = X_temp_df.iloc[split_idx:].reset_index(drop=True)

In [16]:
df_test['anomaly_score'] = test_errors
df_test['anomalous'] = anomaly_flags

### Sort by anomaly score

In [17]:
df_anomalies_ranked = df_test.sort_values(by='anomaly_score', ascending=False)

# Display the top 10 most anomalous records
df_anomalies_ranked.head(10)

,src_ip,dst_ip,bytes_out,bytes_in,hour_of_day,label,anomaly_score,anomalous
88,10.66.110.75,10.249.240.225,27440,44504,7,0,0.009384,1
296,10.68.42.97,10.49.212.3,18795,42275,7,0,0.009367,1
561,10.144.16.93,10.81.52.230,32575,18028,1,0,0.007328,1
567,10.133.85.41,10.42.141.203,280,18899,5,0,0.005766,1
904,10.35.50.188,10.146.80.103,32124,32650,23,0,0.005658,1
574,10.146.25.206,10.172.251.9,9825,20478,14,0,0.005011,1
255,10.223.253.27,10.217.99.60,28199,41141,1,0,0.004394,1
42,10.179.131.130,10.140.20.7,887,19656,9,0,0.003783,1
509,10.96.135.189,10.18.35.111,740,28868,18,0,0.002104,1
857,10.93.210.139,10.216.171.204,22339,15290,17,0,0.001573,1
